In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plot
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

raw_data= pd.read_csv('../input/creditcard.csv')

**Check if classes are imbalanced**

In [ ]:
count_classes = pd.value_counts(raw_data['Class'], sort = True).sort_index()
count_classes.plot(kind = 'bar')
plot.title("Fraud class histogram")
plot.xlabel("Class")
plot.ylabel("Frequency")

In [ ]:
from sklearn.preprocessing import StandardScaler

raw_data['normAmount'] = StandardScaler().fit_transform(raw_data['Amount'].values.reshape(-1, 1))
data = raw_data.drop(['Time','Amount'],axis=1)
data.head()

In [ ]:
X = data.ix[:, data.columns != 'Class']
y = data.ix[:, data.columns == 'Class']

In [ ]:
# Number of data points in the minority class
number_records_fraud = len(data[data.Class == 1])
fraud_indices = np.array(data[data.Class == 1].index)

nonfraud_indices = data[data.Class == 0].index

random_nonfraud_indices = np.random.choice(nonfraud_indices, number_records_fraud, replace = False)
random_nonfraud_indices = np.array(random_nonfraud_indices)

# concatenate everything together
sampled_indices = np.concatenate([fraud_indices,random_nonfraud_indices])

# Under sample dataset
sampled_data = data.iloc[sampled_indices,:]

X_sampled = sampled_data.ix[:, sampled_data.columns != 'Class']
y_sampled = sampled_data.ix[:, sampled_data.columns == 'Class']

# Showing ratio
print("Percentage of normal transactions: ", len(sampled_data[sampled_data.Class == 0])/len(sampled_data))
print("Percentage of fraud transactions: ", len(sampled_data[sampled_data.Class == 1])/len(sampled_data))
print("Total number of transactions in resampled data: ", len(sampled_data))

**Divide into training and test**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sampled,y_sampled,test_size = 0.3, random_state = 0)

fit a decision tree

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

not good enough for a fraud detection system

In [ ]:
clf.get_params()

    **Try logisitic regression with k fold validation**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report, precision_score

In [ ]:
def printing_Kfold_scores(x_train_data,y_train_data,c_param):
    fold = KFold(len(y_train_data),5,shuffle=False) 
    
    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score', 'Mean '])
    # the k-fold will give 2 lists: train_indices = indices[0], test_indices = indices[1]
    j = 0
    for c_param in c_param_range:
        print('C parameter: ', c_param)
        
        recall_accs = []
        for iteration, indices in enumerate(fold,start=1):

            # Call the logistic regression model with a certain C parameter
            lr = LogisticRegression(C = c_param, penalty = 'l1')

            # Use the training data to fit the model. In this case, we use the portion of the fold to train the model
            # with indices[0]. We then predict on the portion assigned as the 'test cross validation' with indices[1]
            lr.fit(x_train_data.iloc[indices[0],:],y_train_data.iloc[indices[0],:].values.ravel())

            # Predict values using the test indices in the training data
            y_pred_undersample = lr.predict(x_train_data.iloc[indices[1],:].values)

            # Calculate the recall score and append it to a list for recall scores representing the current c_parameter
            recall_acc = recall_score(y_train_data.iloc[indices[1],:].values,y_pred_undersample)
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': recall score = ', recall_acc)

        # The mean value of those recall scores is the metric we want to save and get hold of.
        results_table.ix[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Mean recall score ', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].idxmax()]['C_parameter']
    
    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Best model to choose from cross validation is with C parameter = ', best_c)
    print('*********************************************************************************')
    
    return best_c

In [ ]:
best = printing_Kfold_scores(X_train,y_train_data,c_param)

In [ ]:
lr.fit(X_train,y_train.values.ravel())
y_pred_proba = lr.predict_proba(X_train)
    
precision_recall_curve(y_train, y_pred_proba)

In [ ]:
# Call the logistic regression model with the best C parameter w/ to recall
lr = LogisticRegression(C = 0.01, penalty = 'l1')

lr.fit(X_train,y_train.values.ravel())
y_pred_undersample = lr.predict(X_train.values)

In [ ]:
precision, recall, thresholds =  precision_recall_curve(y_train, y_pred_undersample)
plt.plot(recall, precision, color='navy', label='Precision-Recall curve')